<a href="https://colab.research.google.com/github/Ibrahim-Ezmazy/python-Projects/blob/main/MobileNet_CoffeeDiseases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import precision_score, recall_score, f1_score


In [2]:
# Mount Google Drive if needed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import os
import shutil
import random

# Define directories
dataset_dir = "/content/drive/MyDrive/dataset/data"
train_dir = "/content/drive/MyDrive/training"
validation_dir = "/content/drive/MyDrive/validation"

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

# Define the ratio of data to be used for validation
validation_ratio = 0.2  # For example, 20% of the data for validation

# List the classes in the dataset
classes = os.listdir(dataset_dir)

# Loop through each class
for class_name in classes:
    class_dir = os.path.join(dataset_dir, class_name)
    files = os.listdir(class_dir)
    random.shuffle(files)  # Shuffle the files to ensure randomness

    # Calculate the number of samples for validation
    num_validation_samples = int(len(files) * validation_ratio)

    # Split the files into training and validation sets
    validation_files = files[:num_validation_samples]
    train_files = files[num_validation_samples:]

    # Copy or move files to their respective directories
    for file_name in validation_files:
        source = os.path.join(class_dir, file_name)
        destination = os.path.join(validation_dir, class_name, file_name)
        os.makedirs(os.path.join(validation_dir, class_name), exist_ok=True)
        shutil.copy(source, destination)  # Copy file to validation directory

    for file_name in train_files:
        source = os.path.join(class_dir, file_name)
        destination = os.path.join(train_dir, class_name, file_name)
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        shutil.copy(source, destination)  # Copy file to training directory


In [12]:
# Define image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Path to your dataset folders
train_data_dir = '/content/drive/MyDrive/training'
val_data_dir = '/content/drive/MyDrive/validation'


In [13]:

# Define constants
batch_size = 20
num_classes = 2
epochs = 10

# Create image generators
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(150, 150),  # MobileNet input size
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 419 images belonging to 2 classes.
Found 111 images belonging to 2 classes.


In [14]:
# Load MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False)

# Add custom layers on top of MobileNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/10
20/20 [==============================] - 75s 4s/step - loss: 0.7300 - accuracy: 0.6717 - val_loss: 0.3481 - val_accuracy: 0.8300
Epoch 2/10
20/20 [==============================] - 61s 3s/step - loss: 0.1941 - accuracy: 0.9223 - val_loss: 0.2036 - val_accuracy: 0.9000
Epoch 3/10
20/20 [==============================] - 62s 3s/step - loss: 0.0965 - accuracy: 0.9674 - val_loss: 0.1634 - val_accuracy: 0.9500
Epoch 4/10
20/20 [==============================] - 67s 3s/step - loss: 0.0760 - accuracy: 0.9724 - val_loss: 0.1083 - val_accuracy: 0.9700
Epoch 5/10
20/20 [==============================] - 67s 3s/step - loss: 0.0507 - accuracy: 0.9875 - val_loss: 0.0939 - val_accuracy: 0.9800
Epoch 6/10
20/20 [==============================] - 67s 3s/step - loss: 0.0403 - accuracy: 0.9925 - val_loss: 0.0911 - val_accuracy: 0.9600
Epoch 7/10
20/20 [==============================] - 67s 3s/step - loss: 0.0337 - accuracy: 0.9975 - val_loss: 0.0806 - val_accuracy: 0.9700
Epoch 8/10
20/20 [==

In [16]:
# Evaluate the model
evaluation = model.evaluate(validation_generator)
print("Test Accuracy:", evaluation[1])

6/6 [==============================] - 13s 2s/step - loss: 0.0692 - accuracy: 0.9730
Test Accuracy: 0.9729729890823364


In [17]:
# Calculate predictions
validation_generator.reset()  # Reset the generator to avoid shuffling issues
y_pred = model.predict(validation_generator)
y_true = validation_generator.classes

# Convert predictions to class labels
y_pred_classes = tf.argmax(y_pred, axis=1)

# Calculate precision, recall, and F-score
precision = precision_score(y_true, y_pred_classes)
recall = recall_score(y_true, y_pred_classes)
f_score = f1_score(y_true, y_pred_classes)

print("Precision:", precision)
print("Recall:", recall)
print("F-score:", f_score)


6/6 [==============================] - 14s 2s/step
Precision: 0.5625
Recall: 0.5714285714285714
F-score: 0.5669291338582677


In [19]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[20 28]
 [27 36]]
